# Heroes of Pymoli Data Analysis

- Of the 780 total purchases made by our 576 players to date, the majority (84%) of purchases were made by male players, compared to only 14% of total purchases that were made by female players.
- Players in the age range of 20-24 are our leading age demographic, accounting for 47% of the total purchase revenue, while 15-19 (17%) and 25-29 (12%) are the next most profitable age demographics.
- Item #178 (Oathbreaker, Last Hope of the Breaking Storm") is both the most frequently purchased item, and the most profitable.


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
#Player Count
total_players=(purchase_data["SN"].unique())
player_count=len(total_players)
player_count

576

In [3]:
#Purchase Count
total_purchases=purchase_data["Purchase ID"].count()
print(total_purchases)

780


In [4]:
#calculations for purchasing analysis
unique_items=len(purchase_data["Item Name"].unique())
avg_price=purchase_data["Price"].mean()
total_revenue=purchase_data["Price"].sum()

In [5]:
#Purchasing Analysis (Total)
summary=[("Number of Unique Items", [unique_items]), 
         ("Average Price", [avg_price]), 
         ("Number of Purchases", [total_purchases]), 
         ("Total Revenue", [total_revenue])]
df_summary=pd.DataFrame.from_items(summary)
df_summary["Average Price"]=df_summary["Average Price"].map("${:,.2f}".format)
df_summary["Total Revenue"]=df_summary["Total Revenue"].map("${:,.2f}".format)
df_summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [6]:
#Gender Demographics
xy=purchase_data["Gender"].value_counts()
total_gender=xy.sum()
perc_gender=xy/total_gender *100

demo_df=pd.concat([perc_gender, xy], axis=1)
demo_df.columns=["Percentage of Players", "Total Counts"]
demo_df["Percentage of Players"]=demo_df["Percentage of Players"].map("{:,.2f}%".format)
demo_df

,Percentage of Players,Total Counts
Male,83.59%,652
Female,14.49%,113
Other / Non-Disclosed,1.92%,15


In [7]:
#Purchasing Analysis (Gender)
grouped_df=purchase_data.groupby(["Gender"])
gender_spend=grouped_df["Price"].sum()
avg_purchase=gender_spend/xy
pag_df=pd.concat([xy, avg_purchase, gender_spend], axis=1)
pag_df.columns=["Purchase Counts", "Average Purchase Price", "Total Purchase Values"]
pag_df["Total Purchase Values"]=pag_df["Total Purchase Values"].map("${:,.2f}".format)
pag_df["Average Purchase Price"]=pag_df["Average Purchase Price"].map("${:,.2f}".format)
pag_df.index.name="Gender"
pag_df

,Purchase Counts,Average Purchase Price,Total Purchase Values
Gender,,,
Female,113,$3.20,$361.94
Male,652,$3.02,"$1,967.64"
Other / Non-Disclosed,15,$3.35,$50.19


In [8]:
# Age Demographics
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
purchase_data["Age Group"]=pd.cut(purchase_data["Age"], age_bins, labels=group_names)
age_grouped_df=purchase_data.groupby(["Age Group"])

age_purchases=age_grouped_df["Age"].count()
age_totals=age_grouped_df["Price"].sum()
age_averages=age_totals/age_purchases
grand_total=age_totals.sum()
demo_pct=age_totals/grand_total *100
age_demo_df=pd.concat([age_purchases, age_averages, age_totals, demo_pct], axis=1)
age_demo_df.columns=["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Percent of Total Value"]
age_demo_df["Average Purchase Price"]=age_demo_df["Average Purchase Price"].map("${:,.2f}".format)
age_demo_df["Total Purchase Value"]=age_demo_df["Total Purchase Value"].map("${:,.2f}".format)
age_demo_df["Percent of Total Value"]=age_demo_df["Percent of Total Value"].map("{:,.2f}%".format)
age_demo_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Percent of Total Value
Age Group,,,,
<10,23,$3.35,$77.13,3.24%
10-14,28,$2.96,$82.78,3.48%
15-19,136,$3.04,$412.89,17.35%
20-24,365,$3.05,"$1,114.06",46.81%
25-29,101,$2.90,$293.00,12.31%
30-34,73,$2.93,$214.00,8.99%
35-39,41,$3.60,$147.67,6.21%
40+,13,$2.94,$38.24,1.61%


In [9]:
#Top Spenders
user_df=purchase_data.groupby(["SN"])
user_count=user_df["Purchase ID"].count()
user_total=user_df["Price"].sum()
user_avg=user_total/user_count
user_summary=pd.concat([user_count, user_avg, user_total], axis=1)
user_summary.columns=["Purchase Count","Average Purchase Price", "Total Purchase Value"]
user_summary=user_summary.sort_values("Total Purchase Value", ascending=False)
user_summary["Total Purchase Value"]=user_summary["Total Purchase Value"].map("${:,.2f}".format)
user_summary["Average Purchase Price"]=user_summary["Average Purchase Price"].map("${:,.2f}".format)
user_summary.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [10]:
#Most Popular Items
pop_items=purchase_data[["Item ID", "Item Name", "Price"]]
pop_items_grp=pop_items.groupby(["Item ID", "Item Name"])

pop_count=pop_items_grp["Item ID"].count()
pop_totals=pop_items_grp["Price"].sum()
pop_price=pop_totals/pop_count
pop_df=pd.concat([pop_count, pop_price, pop_totals], axis=1)
pop_df.columns=["Purchase Count", "Item Price", "Total Purchase Value"]
sort_df=pop_df.sort_values("Purchase Count", ascending=False)
sort_df["Item Price"]=sort_df["Item Price"].map("${:,.2f}".format)
sort_df["Total Purchase Value"]=sort_df["Total Purchase Value"].map("${:,.2f}".format)
sort_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [11]:
#Most Profitable Items
profit_df=pop_df.sort_values("Total Purchase Value", ascending=False)
profit_df["Item Price"]=profit_df["Item Price"].map("${:,.2f}".format)
profit_df["Total Purchase Value"]=profit_df["Total Purchase Value"].map("${:,.2f}".format)
profit_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
